1. 순수 파이썬의 상대적으로 저조한 성능
2. 최적화되지 않은 라이브러리들의 함정
3. 불완전한 쓰레딩 지원(GIL)
4. 느슨한 타입 시스템과 그로 인한 런타임 에러에 대한 노출

In [4]:
%%time
import pandas as pd

df = pd.read_csv('./StockData/102110.csv',index_col=0, parse_dates=True)
df

Wall time: 2.73 s
Parser   : 263 ms


In [8]:
%%time
# 날짜는 iso 8601 기준으로 표시됨
# pip install ciso8601 해줘야함
# 아래와 같은 parse를 pd.read_csv의 date_parser 에 인자로 제공하면, 데이터를 읽어오는 데 걸리는 시간이 매우 절약됩니다.

import ciso8601
import datetime as dt

def my_parser(date_str):
    return ciso8601.parse_datetime(date_str)

df = pd.read_csv("./StockData/102110.csv", index_col='Date', parse_dates=['Date'], date_parser=my_parser)
df

Wall time: 243 ms


In [9]:
%%time
# 가장 일반적인 slicing code
sliced = df.loc['2020-01-03': '2020-01-08']
# 이 코드의 가장 큰 문제점은, pandas.DatetimeIndex를 통해 ‘시간’으로 DataFrame을 slicing하고 있다는 점임
# 주의할 점은, df.loc 과 datetime의 조합을 사용하는 경우, slice의 뒷부분은 2020–01–08의 00시00분의 열린 괄호가 아닌, 2020–01–08 23시 59분 59초 99… 의 닫힌 괄호입니다. 따라서 2020–01–09 직전까지의 데이터들이 수집됩니다.

Wall time: 685 ms


In [14]:
%%time

# 만약 당신이 데이터에서 날짜를 필요로 하는 이유가 단순하게 데이터 행들의 순서를 보존하기 위해서이고, 다른 복잡한 시간 연산(ex. ‘다음 월말 데이터 찾기’, ‘1시간 후 날짜 찾기’ 등)이 필요한 것이 아니라면 굳이 DatetimeIndex를 사용할 필요가 없습니다. 단순 정렬된 string 만으로도 indexing/slicing이 가능하기 때문입니다.

# 이미 정렬되어 있는 배열을 대상으로 탐색할 때는 이진탐색(binary search)를 사용하면 계산효율을 대폭 높일 수 있다. 
# Numpy는 np.searchsorted(array,value,side='left')를 통해 이진탐색을 수행할 수 있다.

# DatetimeIndex 에서 loc을 통해 slicing하는 연산은 ‘첫 시행’에서 급격한 성능 저하를 야기합니다. 2번째 이후 시행부터는 꽤 괜찮은 성능을 보이지만, searchsorted를 통해 number index 를 직접 찾아서 slicing하는 것보다는 낮은 성능을 보입니다.

import numpy as np
df = pd.read_csv('./StockData/102110.csv')
df

Wall time: 59.8 ms


In [16]:
%%time

idxs = np.searchsorted(df['Date'], '2020-01-03'), np.searchsorted(df['Date'], '2020-01-09')

print(idxs)
df.iloc[idxs[0]: idxs[1]]

(2892, 2896)
Wall time: 998 µs


## Numba와 Cython 을 사용해서 반복문 성능을 향상시키기(fold/scan/rolling_apply 등)

- 금융 시계열을 사용하면서 가장 빈번하게 사용하는 연산 중 하나는 반복(iterative 혹은 sequential-연속) 연산입니다. 과거에서 현재로 행을 순회하면서 진행하는 작업을 의미합니다.